In [1]:
!git clone https://github.com/Lolleeee/MIEEG
import os 
os.chdir("MIEEG")

fatal: destination path 'MIEEG' already exists and is not an empty directory.


In [2]:
from packages.train.training import train_model
from packages.io.file_loader import get_data_loaders
import torch
import os

from packages.data_objects.dataset import TorchDataset, CustomTestDataset

dataset_path = "/kaggle/input/0-5subset-datanooverlap"
# Dummy training loop

dataset = TorchDataset(root_folder=dataset_path, chunk_size=64)

train_loader, val_loader, _ = get_data_loaders(dataset, sets_size={'train': 0.7, 'val': 0.3, 'test': 0.}, norm_axes=(0, 1, 5), batch_size = 64)
# norm_params = train_loader.dataset.dataset._norm_params

Calculating global parameters: 100%|██████████| 87/87 [00:32<00:00,  2.69it/s]

Calculated mean shape: torch.Size([1, 25, 7, 5, 1]), std shape: torch.Size([1, 25, 7, 5, 1])


In [5]:
optimizer = torch.optim.AdamW
from packages.models.vqae import SequenceProcessor
from packages.models.vqae_skip import VQVAE
from packages.train.loss import VQVAELoss, SequenceVQVAELoss
model = SequenceProcessor(chunk_shape=(25, 7, 5, 64), embedding_dim=32, codebook_size=1024, use_quantizer=True)
model.chunk_ae = VQVAE(
    in_channels = 25,
    input_spatial=(7, 5, 64),
    embedding_dim=32,
    codebook_size=1024,
    use_skip_connections=True,
    num_downsample_stages=3,
    skip_strength=0.1)
model.load_state_dict(torch.load("/kaggle/working/MIEEG/model_backups/best_model_epoch_21.pt"))

<All keys matched successfully>

In [21]:
criterion = SequenceVQVAELoss(
    recon_loss_type='mse',
    recon_weight=1.0)    
model.eval()
with torch.no_grad():
    for batch in val_loader:
        out = model(batch)
        print(f"Output shapes: {out[0].shape}, {out[1].shape}, {out[2].shape}, batch: {batch.shape}")
        for i in range(len(out)):
            print(out[i].shape)
        print(criterion(out, batch))
        break

Output shapes: torch.Size([64, 3, 25, 7, 5, 64]), torch.Size([]), torch.Size([192]), batch: torch.Size([64, 3, 25, 7, 5, 64])
torch.Size([64, 3, 25, 7, 5, 64])
torch.Size([])
torch.Size([192])
tensor(0.1829)


In [11]:
out[2]

tensor([529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529,
        529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529,
        529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529,
        529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529,
        529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529,
        529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529,
        529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529,
        529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529,
        529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529,
        529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529,
        529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529,
        529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529,
        529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 529, 5